In [1]:
#list of Import

import os
import random
import requests
import time
import ast
import numpy as np
from glob import glob
import cv2                
from PIL import Image, ImageFile 
import time
from collections import OrderedDict 
import csv
import pandas as pd
import sys
import itertools
import get_results as results

import torch
import torchvision
from torchvision import datasets
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import models
import torchvision.models as models

import time

import matplotlib.pyplot as plt                        
%matplotlib inline

ImageFile.LOAD_TRUNCATED_IMAGES = True

# check if CUDA is available
use_cuda = torch.cuda.is_available()

#pip install torchsummary
from torchsummary import summary

import Helper

# Look at an image

In [2]:
#from PIL import Image
#Image.open(open("data/test/melanoma/ISIC_0012258.jpg", 'rb'))

# Explore data

In [3]:
# load filenames for human and dog images
dataset = np.array(glob("data/*/*/*"))
trainset = np.array(glob("data/train/*/*"))
validset = np.array(glob("data/valid/*/*"))
testset = np.array(glob("data/test/*/*"))
# print number of images in each dataset
print('There are %d total images.' % len(dataset))
print('There are %d total train images.' % len(trainset))
print('There are %d total validation images.' % len(validset))
print('There are %d total test images.' % len(testset))


There are 2750 total images.
There are 2000 total train images.
There are 150 total validation images.
There are 600 total test images.


# Data loader

We prepare three types of transform based on the type of data we get. We want the training data to be augmented whereas we don't want for the Valid and Test datatsets

In [4]:
data_dir = 'data/'

train_transforms = transforms.Compose([
    transforms.RandomRotation(45),
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], 
                         [0.229, 0.224, 0.225])
])

valid_transforms = transforms.Compose([                                 
    transforms.Resize(256),
    transforms.CenterCrop(224),                                                              
    transforms.ToTensor(),                                    
    transforms.Normalize([0.485, 0.456, 0.406], 
                         [0.229, 0.224, 0.225])
])

test_transforms = transforms.Compose([                                 
    transforms.Resize(256),
    transforms.CenterCrop(224),                                                              
    transforms.ToTensor(),                                    
    transforms.Normalize([0.485, 0.456, 0.406], 
                         [0.229, 0.224, 0.225])
])

image_datasets = {x: datasets.ImageFolder(data_dir + x, transform = train_transforms if x == 'train' else valid_transforms if x == 'valid' else test_transforms) for x in ['train', 'valid', 'test']}

loaders_scratch = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=64, shuffle=True) for x in ['train', 'valid', 'test']}


In [5]:
print(image_datasets['test'])

Dataset ImageFolder
    Number of datapoints: 600
    Root location: data/test
    StandardTransform
Transform: Compose(
               Resize(size=256, interpolation=PIL.Image.BILINEAR)
               CenterCrop(size=(224, 224))
               ToTensor()
               Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
           )


## Image vizualisation


In [6]:
# Visualize different training images

def imshow(image, ax=None, title=None, normalize=True):
  """Imshow for Tensor."""
  if ax is None:
    fig, ax = plt.subplots()
  image = image.numpy().transpose((1, 2, 0))
  
  if normalize:
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    image = std * image + mean
    image = np.clip(image, 0, 1)
    
  ax.imshow(image)
  ax.spines['top'].set_visible(False)
  ax.spines['right'].set_visible(False)
  ax.spines['left'].set_visible(False)
  ax.spines['bottom'].set_visible(False)
  ax.tick_params(axis='both', length=0)
  ax.set_xticklabels('')
  ax.set_yticklabels('')
  return ax

images, labels = next(iter(loaders_scratch['train']))
imshow(images[7])

# Build model

In [7]:
# Check model for transfer learning

model_transfer = models.vgg16(pretrained=True)

print(model_transfer.classifier[6].in_features) 
print(model_transfer.classifier[6].out_features)

# Freeze training for all "features" layers
for param in model_transfer.features.parameters():
    param.requires_grad = False
    
n_inputs = model_transfer.classifier[6].in_features

# add last linear layer (n_inputs -> 3 type)
# new layers automatically have requires_grad = True
last_layer = nn.Linear(n_inputs, 3)

model_transfer.classifier[6] = last_layer

# check to see that your last layer produces the expected number of outputs
print(model_transfer.classifier[6].out_features)

if use_cuda:
    model_transfer = model_transfer.cuda()

4096
1000
3


# Train Classifier for VGG16

In [8]:
def train(n_epochs, loaders, model, optimizer, criterion, use_cuda, save_path):
    """returns trained model"""
    # initialize tracker for minimum validation loss
    start = time.time()
    valid_loss_min = np.Inf 
    
    for epoch in range(1, n_epochs+1):
        # initialize variables to monitor training and validation loss
        train_loss = 0.0
        valid_loss = 0.0
        
        ###################
        # train the model #
        ###################
        model.train()
        for batch_idx, (data, target) in enumerate(loaders['train']):
            # move to GPU
            if use_cuda:
                data, target = data.cuda(), target.cuda()
            ## find the loss and update the model parameters accordingly
            ## record the average training loss, using something like
            ## train_loss = train_loss + ((1 / (batch_idx + 1)) * (loss.data - train_loss))
            optimizer.zero_grad()
            # forward pass: compute predicted outputs by passing inputs to the model
            output = model(data)
            # calculate the batch loss
            loss = criterion(output, target)
            # backward pass: compute gradient of the loss with respect to model parameters
            loss.backward()
            # perform a single optimization step (parameter update)
            optimizer.step()
            #update training loss
            train_loss += loss.item()*data.size(0)
            
        ######################    
        # validate the model #
        ######################
        model.eval()
        for batch_idx, (data, target) in enumerate(loaders['valid']):
            # move to GPU
            if use_cuda:
                data, target = data.cuda(), target.cuda()
            ## update the average validation loss
            output = model(data)
            # calculate the batch loss
            loss = criterion(output, target)
            # update average validation loss 
            valid_loss += loss.item()*data.size(0)
            
        # calculate average losses
        train_loss = train_loss/len(loaders['train'].sampler)
        valid_loss = valid_loss/len(loaders['valid'].sampler) 

            
        # print training/validation statistics 
        print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f} \tTime: {:.3f}'.format(
            epoch, 
            train_loss,
            valid_loss,
            time.time() - start
            ))
        start = time.time()
        
        ## TODO: save the model if validation loss has decreased
        if valid_loss <= valid_loss_min and epoch > (n_epochs - 5):
            print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
            valid_loss_min,
            valid_loss))
            torch.save(model.state_dict(), save_path)
            valid_loss_min = valid_loss
            
    # return trained model
    return model

## loss function and optim for VGGG16

In [9]:
criterion_transfer = nn.CrossEntropyLoss()
optimizer_transfer = optim.SGD(model_transfer.classifier.parameters(), lr=0.001)

In [ ]:
# train the model
n_epochs = 3

loaders_transfer = loaders_scratch

model_transfer =  train(n_epochs, 
                        loaders_transfer, 
                        model_transfer, 
                        optimizer_transfer, 
                        criterion_transfer, 
                        use_cuda, 
                        'model_transfer.pt')

In [ ]:
# load the model that got the best validation accuracy (uncomment the line below)
model_transfer.load_state_dict(torch.load('model_transfer.pt'))


In [ ]:
summary(model_transfer,input_size=(3, 224, 224))

In [ ]:
def test(loaders, model, criterion, use_cuda):

    # monitor test loss and accuracy
    test_loss = 0.
    correct = 0.
    total = 0.

    model.eval()
    for batch_idx, (data, target) in enumerate(loaders['test']):
        # move to GPU
        if use_cuda:
            data, target = data.cuda(), target.cuda()
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(data)
        # calculate the loss
        loss = criterion(output, target)
        # update average test loss 
        test_loss = test_loss + ((1 / (batch_idx + 1)) * (loss.data - test_loss))
        # convert output probabilities to predicted class
        pred = output.data.max(1, keepdim=True)[1]
        # compare predictions to true label
        correct += np.sum(np.squeeze(pred.eq(target.data.view_as(pred))).cpu().numpy())
        total += data.size(0)
            
    print('Test Loss: {:.6f}\n'.format(test_loss))

    print('\nTest Accuracy: %2d%% (%2d/%2d)' % (
        100. * correct / total, correct, total))

In [ ]:
test(loaders_scratch, model_transfer, criterion_transfer, use_cuda)

In [ ]:
class_names = [item.replace("_", " ") for item in image_datasets['train'].classes]

def mole_id(img_path):
    # load the image and return the predicted breed
    #image_tensor = image_to_tensor(img_path)
    
    img = Image.open(img_path).convert('RGB')
    transformations = transforms.Compose([transforms.Resize(size=224),
                                          transforms.CenterCrop((224,224)),
                                         transforms.ToTensor(),
                                         transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                                              std=[0.229, 0.224, 0.225])])
    image_tensor = transformations(img)[:3,:,:].unsqueeze(0)
    
    if use_cuda:
        image_tensor = image_tensor.cuda()

    # get sample result
    result = model_transfer(image_tensor)
    # convert output probabilities to predicted class
    _, preds_tensor = torch.max(result, 1)
    pred = np.squeeze(preds_tensor.numpy()) if not use_cuda else np.squeeze(preds_tensor.cpu().numpy())
    
    return class_names[pred]

In [ ]:
img_path = "test/test.jpg"

print(mole_id(img_path))
#image = Image.open(img_path)
img = Image.open(img_path).convert('RGB')
transformations = transforms.Compose([transforms.Resize(size=224), transforms.CenterCrop((224,224))])
image = transformations(img)
plt.imshow(image)
plt.show()

In [ ]:
img_path = "data/test/melanoma/ISIC_0013472.jpg"

print(mole_id(img_path))
#image = Image.open(img_path)
img = Image.open(img_path).convert('RGB')
transformations = transforms.Compose([transforms.Resize(size=224), transforms.CenterCrop((224,224))])
image = transformations(img)
plt.imshow(image)
plt.show()

In [ ]:
img_path = "test/test2_2.jpg"

print(mole_id(img_path))
#image = Image.open(img_path)
img = Image.open(img_path).convert('RGB')
transformations = transforms.Compose([transforms.Resize(size=224), transforms.CenterCrop((224,224))])
image = transformations(img)
plt.imshow(image)
plt.show()

## Test on the whole melanoma test database - no need to run until improved model


In [ ]:
res = 0
x = 0
for x in range(len(testset)):
    
    if mole_id(testset[x]) == "Melanoma":
        res += 1
        
print(res)

# Evaluate model

The model is ranked according to three separate categories:

- Category 1 (Score 1): ROC AUC for melanoma classification.

    This is the ability to distiguish between malignant melanoma and the benign skin lesions (nevus, seborrheic keratosis) by calculating the area under the receiver operating characteristic curve (ROC AUC)[https://scikit-learn.org/stable/modules/generated/sklearn.metrics.roc_auc_score.html] corresponding to this binary classification task.

- Category 2 (Score 2): ROC AUC for melanocytic classification:

    All of the skin lesions that we will examine are caused by abnormal growth of either melanocytes or keratinocytes, which are two different types of epidermal skin cells. Melanomas and nevi are derived from melanocytes, whereas seborrheic keratoses are derived from keratinocytes. The second caterory tests the ability to distinguish between melanocytic and keratinocytic skin lesions.

- Category 3 (Score 3): Mean ROC AUC:

    This catefory take the average of the ROC AUC values from the first two categories.

# load the model

In [ ]:
state_dict = torch.load('model_transfer.pt')
model_transfer.load_state_dict(state_dict)

## Get predictions

In [ ]:
#img_paths = [line.rstrip('\n') for line in open('paths.txt')] 

#print(len(img_paths))

In [ ]:
img_paths = [line.rstrip('\n') for line in testset]

print(len(img_paths))

In [ ]:
print(img_paths[0])

In [ ]:
print("first five: ", img_paths[:5], "\nMiddle five: ", img_paths[300:305], "\nLast five: ", img_paths[-5:])

In [ ]:
def load_image(path):
  image = Image.open(path)
  image = test_transforms(image)
  image = torch.tensor(image, requires_grad=True)
  image = image.unsqueeze(0)
  return image

def get_predictions(img_paths):
  """ Is melanoma? """
  pred_rank1 = [] # Is melanoma?
  pred_rank2 = [] # Is seborrheic?
  for img_path in img_paths:
    img = load_image(img_path)
    #img.to(device)
    prediction = model_transfer(img)
    #img.to("cpu")
    pred_rank1.append(float(prediction.data[0][0]))
    pred_rank2.append(float(prediction.data[0][2]))
  return pred_rank1, pred_rank2

#model.to("cpu")
#model.eval()
print("")

In [ ]:
task1, task2 = get_predictions(img_paths)

In [ ]:
print("Task 1 predictions: ", len(task1))
print("first five predictions (Task 1): ", task1[:5])

print("Task 2 predictions: ", len(task2))
print("first five predictions (Task 2): ", task2[:5])

In [ ]:
with open('results.csv', 'w', newline='') as csvfile:
  doc = csv.writer(csvfile)
  doc.writerow(['Id', 'task_1', 'task_2'])
  for i in range(len(img_paths)):
    doc.writerow([img_paths[i], task1[i], task2[i]])


In [ ]:
from sklearn.metrics import roc_curve, auc, confusion_matrix
threshold = 0.5 

# get ground truth labels for test dataset
truth = pd.read_csv('ground_truth.csv')
y_true = truth.as_matrix(columns=["task_1", "task_2"])

# get model predictions for test dataset
y_pred = pd.read_csv('results.csv')
y_pred = y_pred.as_matrix(columns=["task_1", "task_2"])

# plot ROC curves and print scores
results.plot_roc_auc(y_true, y_pred)
# plot confusion matrix
classes = ['benign', 'malignant']
results.plot_confusion_matrix(y_true[:,0], y_pred[:,0], threshold, classes)